In [116]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [117]:
X,y = make_classification(n_samples=10000, n_features=10,n_informative=3)

In [118]:
print(X.shape)
print(y.shape)

(10000, 10)
(10000,)


In [119]:
X[0]

array([ 0.36151952,  0.5858373 ,  3.08299077, -0.37086497, -0.05560213,
        2.80328629,  0.6926043 ,  0.03629541,  0.09353443,  0.43765548])

In [120]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [121]:
print(X_train.shape)
print(y_train.shape)

print("*"*10)
print(X_test.shape)
print(y_test.shape)

(8000, 10)
(8000,)
**********
(2000, 10)
(2000,)


In [122]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train,y_train)

DecisionTreeClassifier(random_state=42)

In [123]:
y_pred = dt.predict(X_test)

print("Decision Tree accuracy",accuracy_score(y_test,y_pred))

Decision Tree accuracy 0.861


### Bagging

In [124]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.5,
    bootstrap=True,
    random_state=42
)

In [125]:
bag.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=500, random_state=42)

In [126]:
y_pred = bag.predict(X_test)
accuracy_score(y_test, y_pred)

0.901

In [127]:
# diff samples of 500 trees
#bag.estimators_samples_

In [128]:
# no. of samples in 1st bootstrap
bag.estimators_samples_[0].shape

(4000,)

In [129]:
# no. of features in 1st bootstrap
bag.estimators_features_[0].shape

(10,)

### Bagging using SVM

In [130]:
bag = BaggingClassifier(
    estimator=SVC(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    random_state=42
)

bag.fit(X_train,y_train)

BaggingClassifier(estimator=SVC(), max_samples=0.25, n_estimators=500,
                  random_state=42)

In [131]:
y_pred = bag.predict(X_test)
print("Bagging using SVM :", accuracy_score(y_test,y_pred))

Bagging using SVM : 0.884


### Pasting

In [132]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=False,
    random_state=42,
    verbose = 1,
    n_jobs=-1
)

bag.fit(X_train,y_train)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   16.2s finished


BaggingClassifier(bootstrap=False, estimator=DecisionTreeClassifier(),
                  max_samples=0.25, n_estimators=500, n_jobs=-1,
                  random_state=42, verbose=1)

In [133]:
y_pred = bag.predict(X_test)
print("Pasting classifier :",accuracy_score(y_test,y_pred))

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Pasting classifier : 0.9


[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.3s finished


### Random Subspaces

In [134]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=1.0,
    bootstrap=False,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

bag.fit(X_train,y_train)

BaggingClassifier(bootstrap=False, bootstrap_features=True,
                  estimator=DecisionTreeClassifier(), max_features=0.5,
                  n_estimators=500, random_state=42)

In [135]:
y_pred = bag.predict(X_test)
print("Random Subspaces classifier :",accuracy_score(y_test,y_pred))

Random Subspaces classifier : 0.895


In [136]:
bag.estimators_samples_[0].shape

(8000,)

In [137]:
bag.estimators_features_[0].shape

(5,)

### Random Patches

In [138]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

bag.fit(X_train,y_train)

BaggingClassifier(bootstrap_features=True, estimator=DecisionTreeClassifier(),
                  max_features=0.5, max_samples=0.25, n_estimators=500,
                  random_state=42)

In [139]:
y_pred = bag.predict(X_test)
print("Random Patches classifier :",accuracy_score(y_test,y_pred))

Random Patches classifier : 0.893


### OOB Score

In [140]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    oob_score=True,
    random_state=42
)

bag.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, oob_score=True, random_state=42)

In [141]:
bag.oob_score_

0.903

In [142]:
y_pred = bag.predict(X_test)
print("Accuracy",accuracy_score(y_test,y_pred))

Accuracy 0.898


### Bagging Tips


1. Bagging generally gives better results than Pasting
2. Good results come around the 25% to 50% row sampling mark
3. Random patches and subspaces should be used while dealing with high dimensional data
4. To find the correct hyperparameter values we can do GridSearchCV/RandomSearchCV

### Applying GridSearchCV

In [143]:
from sklearn.model_selection import GridSearchCV

In [147]:
# parameters = {
#     'n_estimators': [50,100,500],
#     'max_samples': [0.1,0.4,0.7,1.0],
#     'bootstrap' : [True,False],
#     'max_features' : [0.1,0.4,0.7,1.0]
#     }

parameters = {
    'n_estimators': [50,100,500],
    'max_samples': [0.1,0.4],
    'bootstrap' : [True,False],
    }

In [148]:
search = GridSearchCV(BaggingClassifier(), parameters, cv=5)

In [149]:
search.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=BaggingClassifier(),
             param_grid={'bootstrap': [True, False], 'max_samples': [0.1, 0.4],
                         'n_estimators': [50, 100, 500]})

In [150]:
search.best_params_

{'bootstrap': True, 'max_samples': 0.4, 'n_estimators': 500}

In [151]:
search.best_score_

np.float64(0.9035)